In [154]:
# import libraries
from datetime import datetime
from pandas_datareader import data as pdr
import yfinance as yf
import pandas as pd
import time
from selenium import webdriver
from bs4 import BeautifulSoup as bs
import random
import warnings
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException

pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)

In [155]:
stock_symbol = "5071.KL"
ticker = yf.Ticker(stock_symbol)
# Get historical data for the ticker
hist = ticker.history(period="1y")
print(hist)

                           Open  High   Low  Close   Volume  Dividends  \
Date                                                                     
2022-10-06 00:00:00+08:00  1.93  1.95  1.93   1.94   182400        0.0   
2022-10-07 00:00:00+08:00  1.95  1.95  1.91   1.91   211400        0.0   
2022-10-11 00:00:00+08:00  1.91  1.95  1.81   1.86  1002800        0.0   
2022-10-12 00:00:00+08:00  1.85  1.86  1.85   1.86   520000        0.0   
2022-10-13 00:00:00+08:00  1.86  1.91  1.85   1.90   285100        0.0   
2022-10-14 00:00:00+08:00  1.93  1.93  1.86   1.90  3116100        0.0   
2022-10-17 00:00:00+08:00  1.94  1.94  1.87   1.90  2779700        0.0   
2022-10-18 00:00:00+08:00  1.91  1.92  1.89   1.90    69000        0.0   
2022-10-19 00:00:00+08:00  1.91  1.91  1.90   1.90   219300        0.0   
2022-10-20 00:00:00+08:00  1.91  1.93  1.90   1.92   333800        0.0   
2022-10-21 00:00:00+08:00  1.91  1.93  1.88   1.90   162600        0.0   
2022-10-25 00:00:00+08:00  1.92  1.92 

In [156]:
# Assuming hist is a DataFrame with the date set as the index

# Access the date of the first and last record (assuming the index is in datetime format)
# Define the purchase_date and current_date (replace these with your actual dates)
purchase_date = hist.index[0]
current_date = hist.index[-1]
purchase_date = purchase_date.strftime('%Y-%m-%d')
current_date = current_date.strftime('%Y-%m-%d')

# Access the first and last record's Close column
purchase_price = hist['Close'].iloc[0]
current_price = hist['Close'].iloc[-1]

# Now, first_record_date contains the date from the first record
print(purchase_price)


1.940000057220459


In [157]:
price_diff = current_price - purchase_price
price_perc = price_diff / purchase_price * 100
print(price_perc)

-1.0309329662834354


In [142]:
df2 = pd.read_csv('historicaldividends.csv')

# Convert EX Date column to datetime
df2['EX Date'] = pd.to_datetime(df2['EX Date'])
df2['EX Date'] = pd.to_datetime(df2['EX Date'].dt.strftime('%Y-%m-%d'))

# Filter the DataFrame based on the Stock Code and EX Date range
filtered_df2 = df2[(df2['Stock Code'] == stock_symbol[:-3]) & (df2['EX Date'] >= purchase_date) & (df2['EX Date'] <= current_date)]
filtered_df2

# Calculate the sum of the 'Amount' column in the filtered DataFrame
dividend_amount = filtered_df2['Amount'].sum()

# Print the calculated dividend_amount
print(f"Total dividend amount between {purchase_date} and {current_date} for Stock Code {stock_code}: {dividend_amount}")

Total dividend amount between 2022-10-06 and 2023-10-06 for Stock Code 7090: 0.0


In [143]:
dividend_yield = dividend_amount / purchase_price
print(dividend_yield)

0.0


# ---------------------------------------------------------------------------------------------------------------

# Putting it all together in a loop

In [158]:
df = pd.read_csv('exports/filtered_class_a_b.csv')
df2 = pd.read_csv('historicaldividends.csv')

# Convert EX Date column to datetime
df2['EX Date'] = pd.to_datetime(df2['EX Date'])
df2['EX Date'] = pd.to_datetime(df2['EX Date'].dt.strftime('%Y-%m-%d'))

results = []

# Loop through each ticker symbol in the CSV file
for ticker_symbol in df['Ticker']:
    # Construct the ticker object
    ticker = yf.Ticker(ticker_symbol)

    # Get historical data for the ticker
    hist = ticker.history(period="1y")

    purchase_date = hist.index[0]
    current_date = hist.index[-1]
    purchase_date = purchase_date.strftime('%Y-%m-%d')
    current_date = current_date.strftime('%Y-%m-%d')

    # Access the first and last record's Close column
    purchase_price = hist['Close'].iloc[0]
    current_price = hist['Close'].iloc[-1]

    # Now, first_record_date contains the date from the first record
    print(purchase_price)
    
    price_diff = current_price - purchase_price
    price_perc = (current_price - purchase_price) / purchase_price * 100
    print(price_perc)
    
    # Filter the DataFrame based on the Stock Code and EX Date range
    filtered_df2 = df2[(df2['Stock Code'] == ticker_symbol[:-3]) & (df2['EX Date'] >= purchase_date) & (df2['EX Date'] <= current_date)]
    filtered_df2

    # Calculate the sum of the 'Amount' column in the filtered DataFrame
    dividend_amount = filtered_df2['Amount'].sum()

    # Print the calculated dividend_amount
    print(f"Total dividend amount between {purchase_date} and {current_date} for Stock Code {stock_code}: {dividend_amount}")
    
    dividend_yield = dividend_amount / purchase_price
    print(dividend_yield)
    
    # Append the results to the list for this ticker symbol
    results.append({
        'Ticker': ticker_symbol,
        'Purchase Date': purchase_date,
        'Current Date': current_date,
        'Purchase Price': purchase_price,
        'Current Price': current_price,
        'Price Difference': price_diff,
        'Price Percentage Change': price_perc,
        'Dividend Amount': dividend_amount,
        'Dividend Yield': dividend_yield
    })

# Convert the results list to a DataFrame
results_df = pd.DataFrame(results)

# Now, you have the backtesting results for each ticker in the CSV file


2.049363374710083
19.061367414436113
Total dividend amount between 2022-10-06 and 2023-10-06 for Stock Code 7090: 0.05500000000000001
0.026837602681262264
1.7610385417938232
40.825993311158136
Total dividend amount between 2022-10-06 and 2023-10-06 for Stock Code 7090: 0.22
0.12492628342813232
1.940000057220459
-1.0309329662834354
Total dividend amount between 2022-10-06 and 2023-10-06 for Stock Code 7090: 0.0
0.0
1.0300841331481934
11.641363957665874
Total dividend amount between 2022-10-06 and 2023-10-06 for Stock Code 7090: 0.055
0.05339369691280101
0.4518730342388153
94.7449678456687
Total dividend amount between 2022-10-06 and 2023-10-05 for Stock Code 7090: 0.0248
0.05488267305389411
3.1552674770355225
12.193339791876335
Total dividend amount between 2022-10-06 and 2023-10-06 for Stock Code 7090: 0.135
0.042785596144399446
0.3956896662712097
-1.437914876560031
Total dividend amount between 2022-10-06 and 2023-10-06 for Stock Code 7090: 0.01
0.02527233044581432


In [159]:
results_df

,Ticker,Purchase Date,Current Date,Purchase Price,Current Price,Price Difference,Price Percentage Change,Dividend Amount,Dividend Yield
0,7090.KL,2022-10-06,2023-10-06,2.049363,2.44,0.390637,19.061367,0.0550,0.026838
1,5248.KL,2022-10-06,2023-10-06,1.761039,2.48,0.718961,40.825993,0.2200,0.124926
2,5071.KL,2022-10-06,2023-10-06,1.940000,1.92,-0.020000,-1.030933,0.0000,0.000000
3,2062.KL,2022-10-06,2023-10-06,1.030084,1.15,0.119916,11.641364,0.0550,0.053394
4,03047.KL,2022-10-06,2023-10-05,0.451873,0.88,0.428127,94.744968,0.0248,0.054883
5,6139.KL,2022-10-06,2023-10-06,3.155267,3.54,0.384732,12.193340,0.1350,0.042786
6,7285.KL,2022-10-06,2023-10-06,0.395690,0.39,-0.005690,-1.437915,0.0100,0.025272
